In [ ]:
!pip install langchain==0.1.14
!pip install langchain-experimental==0.0.56
!pip install langchain-community==0.0.31
!pip install faiss-cpu==1.8.0
!pip install pdfplumber==0.11.0
!pip install gradio==4.25.0

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("llm-research-paper.pdf")
docs = loader.load()

print("Number of pages in the PDF:",len(docs))

docs[2].page_content

In [52]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

In [53]:
# Check number of chunks created
print("Number of chunks created: ", len(documents))

 # Printing first few chunks
for i in range(len(documents)):
  print()
  print(f"CHUNK : {i+1}")
  print(documents[i].page_content)

Number of chunks created:  80

CHUNK : 1
Mind’s Eye of LLMs: Visualization-of-Thought Elicits
Spatial Reasoning in Large Language Models
WenshanWu† ShaoguangMao† YadongZhang†,‡,∗
YanXia† LiDong† LeiCui† FuruWei†
†MicrosoftResearch ‡EastChinaNormalUniversity
Abstract
Largelanguagemodels(LLMs)haveexhibitedimpressiveperformanceinlanguage
comprehension and various reasoning tasks. However, their abilities in spatial
reasoning, a crucial aspect of human cognition, remain relatively unexplored. Humanpossessaremarkableabilitytocreatementalimagesofunseenobjectsand
actionsthroughaprocessknownastheMind’sEye,enablingtheimaginationof
theunseenworld. Inspiredbythiscognitivecapacity,weproposeVisualization-
of-Thought(VoT)prompting. VoTaimstoelicitspatialreasoningofLLMsby
visualizingtheirreasoningtraces,therebyguidingsubsequentreasoningsteps. We
employedVoTformulti-hopspatialreasoningtasks,includingnaturallanguage
navigation,visualnavigation,andvisualtilingin2Dgridworlds. Experimental
resultsdemonstr

In [54]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()
# Create the vector store
vector = FAISS.from_documents(documents, embedder)

In [55]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [56]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
genai.configure(api_key="API-KEY")

In [63]:
from langchain.llms.base import BaseLLM
from langchain.schema import Generation, LLMResult, Document
from typing import Optional, List, Any
from pydantic import Field
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.schema.retriever import BaseRetriever

# Define the GenAIWrapper
class GenAIWrapper(BaseLLM):
    model: Any = Field(...)  # Explicitly define the model field

    def __init__(self, model):
        """Initialize the wrapper with the underlying GenAI model."""
        super().__init__()
        self.model = model

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Simplified single-shot call method."""
        response = self.model.generate_text(prompt)
        return response.text  # Ensure this returns the text output

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        **kwargs: Any
    ) -> LLMResult:
        """Handles batched prompts and returns results in LangChain's standard format."""
        generations = []
        for prompt in prompts:
            response = self.model.generate_content(prompt)  # Adjust the method name if necessary
            generations.append(Generation(text=response.text))

        # Wrap results in LangChain's LLMResult object
        return LLMResult(generations=[generations])

    @property
    def _llm_type(self) -> str:
        return "genai"



model = genai.GenerativeModel("gemini-1.5-flash")

# Wrap the model using the LangChain-compatible wrapper
wrapped_model = GenAIWrapper(model=model)

# Create prompts and chains
prompt = """
1. Use the following pieces of context to provide a detailed explanation in response to the student's query.
2. If you don't know the answer, say "I don't know" without making up an answer.
3. Make sure to break down the explanation in a clear, student-friendly manner, using simple language. Aim to help the student understand the topic better.
Context: {context}
Question: {question}
Detailed Explanation: """

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)
llm_chain = LLMChain(llm=wrapped_model, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)
combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Test the QA chain
response = qa.invoke({"query": "Explain VoT prompting in basic terms"})
print(response["result"])




> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to provide a detailed explanation in response to the student's query.
2. If you don't know the answer, say "I don't know" without making up an answer.
3. Make sure to break down the explanation in a clear, student-friendly manner, using simple language. Aim to help the student understand the topic better.
Context: Context:
content:Weusep todenoteapre-trainedLMwithparametersθ,x,y,ztodenotealanguagesequence,and
θ
vtodenoteavisualizationsequenceintextform. Inamulti-hopspatialreasoningtaskwithinputx,
CoTpromptinggeneratesaseriesofintermediatestepsz ,··· ,z ,eachstepz ∼p (z |x,z )
1 n i θ i 1···i−1
is sampled sequentially, followed by the output y ∼ p (y|x,z ). As shown in Figure 1, VoT
θ 1···n
promptingenhancesthisprocessbyaddingavisuospatialsketchpadtoeachintermediatestepz ,
i
thenthesubsequentstepz issampledconditionedonpriorstepsz andvisualizationsv .